In [1]:
#This script reads the arpeggio output per complex summary (created in "setup_and_process_arpeggio.ipynb") and defines the residues corresponding to the indicated interaction positions. 
#Note this will reduce the epitope and paratope as determined by arpeggio because some of the residues will be hetero atoms such that no amino acid type can be assigned
#@autor: Henriette Capel
#@data: 11-05-2022

In [2]:
#Import modules
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import ast
from ABDB import database as db
from sklearn import metrics

#Needed to analyse arpeggio
import Bio.PDB
from Bio.Data.IUPACData import protein_letters_3to1
import re
%load_ext nb_black

/data/icarus/capel/miniconda3/envs/minor_internship/lib/python3.10/site-packages/Bio/SubsMat/__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  warnings.warn(


In [3]:
#Constants
columns_types_interactions = ['clash', 'covalent', 'vdw_clash', 'vdw', 'proximal', 'hbond', 'weak_hbond', 'xbond', 'ionic', 'metal_complex', 'aromatic', 'hydrophobic', 'carbonyl', 'polar', 'weak_polar']
interaction_types = ['covalent', 'vdw', 'hbond', 'weak_hbond', 'xbond', 'ionic', 'metal_complex', 'aromatic', 'hydrophobic', 'carbonyl', 'polar', 'weak_polar'] #Check

In [4]:
#Functions
def read_dataset(filename_ds):
    
    df = pd.read_csv(filename_ds, converters={i: str for i in range(100)})
    
    for colname in df.columns.values.tolist():
        try:
            df[colname] = [ast.literal_eval(d) for d in df[colname]]
        except:
            pass

    return df

def add_interaction_residues_to_arpeggio_output(df_my_interactions, df_arp):
    #function to transform the apreggio output dataframe, "df_arp", that was created before into the same format as "df_my_interactions".
    #The epitope definition of "df_my_interactions" is used to assign the correct amino acid to the epitope residues as identified by arpeggio. By calling "reconstruct_interaction_dict".
    #The amino acids were not stored before for the arpeggio output because arpeggio does not provide the amino acid
    
    df_my_select = df_my_interactions[["pdb", "antigen_chain", "antibody_chain", "epitope_MWV_dict", "paratope_MWV_dict"]]
    df_merged_select = pd.merge(df_my_select, df_arp, on=["pdb", "antigen_chain", "antibody_chain"])
    
    list_new_dicts_epi = []
    count_only_arpeggio_epi = 0
    count_failed_epi = 0
    
    list_new_dicts_para = []
    count_only_arpeggio_para = 0
    count_failed_para = 0
    
    for index, row in df_merged_select.iterrows():
        pdb, ag_chain, ab_chain = determine_complex_id(row)
        
        arp_epi_dict = row['epitope_interactions'] #chains of full length seperately
        arp_para_dict = row['paratope_interactions'] #Chains of full length seperately
        my_epi_dict = row['epitope_MWV_dict'] #Chains of full length seperately and combined
        my_para_dict = row['paratope_MWV_dict'] #Chains of full length seperately
    
        if len(ab_chain) ==2: #Full length antibody
            #epitope
            dict_new_epi, count_only_arpeggio_epi_ind, count_failed_epi_ind = determine_epitope_full_length(arp_epi_dict, my_epi_dict, pdb, ag_chain, ab_chain)
            count_only_arpeggio_epi += count_only_arpeggio_epi_ind
            count_failed_epi += count_failed_epi_ind
            
            #Paratope
            dict_new_para, count_only_arpeggio_para_ind, count_failed_para_ind = determine_paratope_full_length(arp_para_dict, my_para_dict, pdb, ab_chain)
            count_only_arpeggio_para += count_only_arpeggio_para_ind
            count_failed_para += count_failed_para_ind
            
            
        else: #Single domain antibody
            #epitope
            dict_new_epi, only_arpeggio_epi, failed_numbers_epi = reconstruct_interaction_dict(arp_epi_dict, my_epi_dict, pdb, ag_chain)
            count_only_arpeggio_epi += only_arpeggio_epi
            count_failed_epi += failed_numbers_epi
            
            #paratope
            dict_new_para, only_arpeggio_para, failed_numbers_para = reconstruct_interaction_dict(arp_para_dict, my_para_dict, pdb, ab_chain)
            count_only_arpeggio_para += only_arpeggio_para
            count_failed_para += failed_numbers_para
            
            
        list_new_dicts_epi.append(dict_new_epi)
        list_new_dicts_para.append(dict_new_para)
    
    #add this list_new_dicts as a column to the arpeggio dataframe. Note that the order of complexes differ from the merged one. So be carefull and merge properly back on the complex information
    df_merged_select = df_merged_select.drop(['epitope_MWV_dict', 'paratope_MWV_dict'], axis=1) #remove the old epitope_MWV_dict column
    
    df_merged_select['epitope_MWV_dict'] = list_new_dicts_epi #Add the new epitope
    df_merged_select['paratope_MWV_dict'] = list_new_dicts_para #Add the new paratope
    
    print(f"Number positions only in arpeggio dict epitope: {count_only_arpeggio_epi}")
    print(f"Number positions failed to find epitope: {count_failed_epi}")
    print(f"Number positions only in arpeggio dict paratope: {count_only_arpeggio_para}")
    print(f"Number positions failed to find paratope: {count_failed_para}")

    return df_merged_select

def determine_epitope_full_length(arp_epi_dict, my_epi_dict, pdb, ag_chain, ab_chain):
    dict_new = {}
    count_only_arpeggio = 0
    count_failed = 0 
    
    #Heavy chain
    dict_new_heavy, only_arpeggio, failed_numbers = reconstruct_interaction_dict(arp_epi_dict[ab_chain[0]], my_epi_dict[ab_chain[0]], pdb, ag_chain)
    count_only_arpeggio += only_arpeggio
    count_failed += failed_numbers
    dict_new[ab_chain[0]] = dict_new_heavy

    #Light chain
    dict_new_light, only_arpeggio, failed_numbers = reconstruct_interaction_dict(arp_epi_dict[ab_chain[1]], my_epi_dict[ab_chain[1]], pdb, ag_chain, dict_new_heavy) #Give dict_new_heavy to reduce computational cost
    count_only_arpeggio += only_arpeggio
    count_failed += failed_numbers
    dict_new[ab_chain[1]] = dict_new_light

    #Heavy and light chain
    dict_new_combined = dict_new_heavy | dict_new_light #Merge without having same key and element twice
    dict_new[ab_chain] =  dict_new_combined
    
    return dict_new, count_only_arpeggio, count_failed

def determine_paratope_full_length(arp_para_dict, my_para_dict, pdb, ab_chain):
    dict_new = {}
    count_only_arpeggio = 0
    count_failed = 0 
    
    #Heavy chain
    dict_new_heavy, only_arpeggio, failed_numbers = reconstruct_interaction_dict(arp_para_dict[ab_chain[0]], my_para_dict[ab_chain[0]], pdb, ab_chain[0])
    count_only_arpeggio += only_arpeggio
    count_failed += failed_numbers
    dict_new[ab_chain[0]] = dict_new_heavy
    
    #light chain
    dict_new_light, only_arpeggio, failed_numbers = reconstruct_interaction_dict(arp_para_dict[ab_chain[1]], my_para_dict[ab_chain[1]], pdb, ab_chain[1]) #Do not give it the paratope dict of the heavy chain as it is a different chain so different amino acids at same numbering positions
    count_only_arpeggio += only_arpeggio
    count_failed += failed_numbers
    dict_new[ab_chain[1]] = dict_new_light
    
    return dict_new, count_only_arpeggio, count_failed
    
    
def reconstruct_interaction_dict(arp_dict, my_dict, pdb, chain, dict_hl_previously_determined = False):
    #This function assigns the correct amino acid to the epitope positions as indicated by arpeggio. This is done by looking up the amino acid in "epi_my_dict".
    #If the position can not be found, call "find_aa_in_pdb". This function will open the pdb to find the amino acid. This is time consuming and therefore only performed for the missing positions.
    #The last argument is an optional argument, which can be used when identifying the epitope for the light chain if the heavy chain is previously determined (or other way around). 
    #It is useful in this case to reduce computational cost. 
    
    dict_new = {}
    failed_in_this_dict = 0
    
    #Be sure both keys are strings and convert my_dict such that insertion is handled correctly
    arp_dict = {str(key): str(value) for key, value in arp_dict.items()}
    my_dict = {transform_my_dict_pos_aa_keys(key): str(value) for key, value in my_dict.items()}
    
    list_positions_not_found = []
    
    for position in arp_dict.keys():
        try:
            dict_new[position] = str(my_dict[position])
        except KeyError:
            list_positions_not_found.append(position)
    
    #find the amino acids of the missing positions for full length antibodies possibly easy if on of the chains has been determined before
    #if the heavy (or light) chain epitope is previously determined use this to find the position in order to reduce computational cost 
    if dict_hl_previously_determined: #do only if given
        list_positions_find_in_pdb = []
        for position_missing in list_positions_not_found:
            try:
                dict_new[position_missing] = str(dict_hl_previously_determined[position_missing])  ##HIER STOND EERST NIET _missing, dat was een fout toch?
            except KeyError:
                list_positions_find_in_pdb.append(position_missing)
                
    else: #if dictionary is not given, it is empty and the list of positions that still need to be find stays the same
        list_positions_find_in_pdb = list_positions_not_found
    
    #find the amino acids of the positions that are still not determined. Do this for the whole list in one go to reduce computational cost
    number_res_only_in_arpeggio = len(list_positions_find_in_pdb)
    
    if len(list_positions_find_in_pdb) != 0: #Do if there exist positions that are not found yet 
        dict_found_positions_aa = find_aa_in_pdb(pdb, chain, list_positions_not_found)
        dict_new = dict_new | dict_found_positions_aa
        failed_in_this_dict = len(list_positions_find_in_pdb) - len(dict_found_positions_aa.keys())
    
    return dict_new, number_res_only_in_arpeggio, failed_in_this_dict

def transform_my_dict_pos_aa_keys(key_input):
    #Transform such that keys like "112_A" will become "112A". This is needed to compare to arpeggio dict.
    try: 
        key_list = key_input.split("_")
        key_output = f"{key_list[0]}{key_list[1]}"
        
    except AttributeError: #does not have an insertion
        key_output= str(key_input)
        
    return key_output

def seperate_position_insertion(position):
    match = re.match(r"([0-9]+)([a-z]+)", position, re.I)
    if match:
        items = match.groups()
        pos_num = int(items[0])
        pos_ins = str(items[1])
    else:
        pos_num = int(position)
        pos_ins = " "    
    return pos_num, pos_ins
            
def find_aa_in_pdb(pdb, chain_given, position_list):
    #This function assigns the correct amino acid to a certain epitope residue position by iterating through the BioPython version of the pdb. 
    dict_found_positions = {}
        
    #Find position by looping over BioPython
    p = db.fetch(pdb)
    imgt_structure = p.get_structure(scheme="imgt", definition = 'imgt')
    for model in imgt_structure:
        model_id = model.get_id()
        for chain_type in model:
            chain_type_id = chain_type.get_id()
            
            #if chain_type_id == "Antigen":
            for chain in chain_type:
                chain_id = chain.get_id()
                if chain_id == chain_given:
                    #Now that we are in the correct chain. Loop over all the positions we need to find
                    for position in position_list:
                        pos_num, pos_ins = seperate_position_insertion(position) #determine the number and the insertion seperately
                        for residue in chain:
                            res_id = residue.get_id()
                            res_position = residue.get_id()[1]
                            res_insertion = residue.get_id()[2]
                            if res_position == pos_num and res_insertion == pos_ins:
                                res_name = residue.resname.strip()
                                try:  
                                    #Get 1 code amino acid name
                                    res_name_3code = res_name[0]+res_name[1:].lower() #Note: should start with upper case letter, all others lower case
                                    res_name_1code = Bio.Data.IUPACData.protein_letters_3to1[res_name_3code]
                                    dict_found_positions[position] = res_name_1code
                                    break #This position is found so go back to loop before and start with the new position
                                except KeyError: 
                                    print(f"Residue 1D code could not be found: {pdb}, {chain_given}, {position}")
                                    print(residue)
                                    break #This position is found but the 1D code could not be determined, so go back to loop before and start with the new position
    return dict_found_positions     

def determine_complex_id(df_row):
    #Determine the pdb, antigen chain, and antibody chain of a given row of a dataframe
    pdb = df_row["pdb"]
    ag_chain = df_row["antigen_chain"]
    ab_chain = df_row["antibody_chain"]

    return pdb, ag_chain, ab_chain


def add_residue_interactions_to_arpeggio_output(df_formatted, columns_types_interactions, interaction_types_list):
    #This function uses the dataframe created by the function "add_interaction_residues_to_arpeggio_output" to determine for which residues we need to know the number of interactions.
    #These number of interactions can be found in the "*_residue_interactions.csv" files which are stored per complex. These files where created in the notebook: "setup_and_process_arpeggio.ipynb".
    #Notel we can not use the column "epitope/paratope_interations" which was created in the same arpeggio notebook, because these include the interactions of the residue with a hetero atom.
    
    #This new dictionary will thus both be smaller than the existing interaction column and the number of interactions will be reduced. 
    #Save in the same way as for my annotations (see "ds_binding_residues.ipynb") so columns are called "eptiope/paratope_num_interactions" and the keys are <aa>_<position_num>_<position_ins>, in which the last one is only included if it is an insertion
    #For full length antibodies the epitope is just one dictionary, the paratope is splitted per chain.
    
    #Determine the number of interactions per residue without hetero atoms:
    df_residue_level_interactions = determine_number_interactions_without_hetero(df_formatted, columns_types_interactions, interaction_types_list)

    #Combine dataframes
    df_formatted_select = df_formatted[['pdb', 'antigen_chain', 'antibody_chain', 'epitope_interactions', 'paratope_interactions', 'epitope_MWV_dict', 'paratope_MWV_dict']]
    df_merged = pd.merge(df_formatted_select, df_residue_level_interactions, on=["pdb", "antigen_chain", "antibody_chain"])
    
    #Make correct format of interaction keys and remove epitope/paratope residues that were only interacting with hetero atoms
    df_correct_format = clean_information_complexes(df_merged)
    
    #Select the columns of interest to return
    df_correct_format = df_correct_format[['pdb', 'antigen_chain', 'antibody_chain', 'epitope_MWV_dict', 'epitope_num_interactions', 'paratope_MWV_dict', 'paratope_num_interactions', 'number_interactions', 'number_contacts', 'clash', 'covalent', 'vdw_clash', 'vdw', 'proximal', 'hbond', 'weak_hbond', 'xbond', 'ionic','metal_complex', 'aromatic', 'hydrophobic', 'carbonyl', 'polar','weak_polar',]]
    
    return df_correct_format

def determine_number_interactions_without_hetero(df_formatted, columns_types_interactions, interaction_types_list):
    #This function opens the *_residue_interactions.csv files of all complexes in the dataframe, removes the residues that are hetero atoms, and thus should not be considered, and counts the number of interactions on the residue level. 
    
    #Set up empty summary dataframe
    columns_ds_df = ['pdb', 'antigen_chain', 'antibody_chain'] + columns_types_interactions
    df_store_interaction_per_dataset = pd.DataFrame(columns= columns_ds_df)

    epitope_num_interactions_list = []
    paratope_num_interactions_list = []
    index_counter_main = 0
    
    for index,row in df_formatted.iterrows():
        pdb, ag_chain, ab_chain = determine_complex_id(row)
        if len(ab_chain) == 2:
            file_name = f"/data/icarus/capel/data_arpeggio/output/full_length/{pdb}_{ag_chain}_{ab_chain}_residue_interactions.csv"
        else:
            file_name = f"/data/icarus/capel/data_arpeggio/output/single_domain/{pdb}_{ag_chain}_{ab_chain}_residue_interactions.csv"
        
        try:
            df_interactions_res = read_dataset(file_name)
            df_interactions_res[['chain_1', 'position_1']] = df_interactions_res['residue_1'].str.split('/', expand=True)
            df_interactions_res[['chain_2', 'position_2']] = df_interactions_res['residue_2'].str.split('/', expand=True)

            if len(ab_chain) == 2: #full length antibody
                #seperate the heavy and light chain
                df_residue_interaction_heavy = df_interactions_res[df_interactions_res["chain_2"] == ab_chain[0]]
                df_residue_interaction_light = df_interactions_res[df_interactions_res["chain_2"] == ab_chain[1]]

                #Remove the residues that were determined as not normal residues
                epitope_pos = list(row["epitope_MWV_dict"][ab_chain].keys())
                paratope_pos_heavy = list(row["paratope_MWV_dict"][ab_chain[0]].keys())
                paratope_pos_light = list(row["paratope_MWV_dict"][ab_chain[1]].keys())
                
                df_interactions_res_selected_heavy = remove_not_found_aa_residues(df_residue_interaction_heavy, epitope_pos, paratope_pos_heavy)
                df_interactions_res_selected_light = remove_not_found_aa_residues(df_residue_interaction_light, epitope_pos, paratope_pos_light)

                #Make the summary 
                dict_summary_per_complex_heavy = store_interactions_residue_per_complex_dict(df_interactions_res_selected_heavy, columns_types_interactions, interaction_types_list)
                dict_summary_per_complex_light = store_interactions_residue_per_complex_dict(df_interactions_res_selected_light, columns_types_interactions, interaction_types_list)

                #Combine
                dict_summary_per_complex = combine_full_length_output(dict_summary_per_complex_heavy, dict_summary_per_complex_light, ab_chain[0], ab_chain[1])
                
                #CHECK
                assert determine_length_interaction(dict_summary_per_complex['epitope_interactions_filtered']) == determine_length_interaction(dict_summary_per_complex['paratope_interactions_filtered'])   
                
            else: #Single domain antibody
                #Remove the residues that were determined as not normal residues
                epitope_pos = list(row["epitope_MWV_dict"].keys())
                paratope_pos = list(row["paratope_MWV_dict"].keys())
                df_interactions_res_selected = remove_not_found_aa_residues(df_interactions_res, epitope_pos, paratope_pos)

                #Make the summary of this complex
                dict_summary_per_complex = store_interactions_residue_per_complex_dict(df_interactions_res_selected, columns_types_interactions, interaction_types_list)
            
            dict_summary_per_complex['pdb'] = pdb
            dict_summary_per_complex['antigen_chain'] = ag_chain
            dict_summary_per_complex['antibody_chain'] = ab_chain
            df_line_info = pd.DataFrame(dict_summary_per_complex, index=[index_counter_main])
            index_counter_main += 1

            #Update dataframe
            df_store_interaction_per_dataset = pd.concat([df_store_interaction_per_dataset, df_line_info], ignore_index = True, axis = 0)
            
        except FileNotFoundError:
            print(f"WARNING FILE NOT FOUND: {pdb}_{ag_chain}_{ab_chain}")
            
    return df_store_interaction_per_dataset


def determine_length_interaction(list_of_chains_interactions):
    #Function to determine the number of interactions based on the format: [{heavy_chain: {position: num_interactions}, light_chain: {position: num_interactions}}]
    len_interaction = 0
    for chain in list_of_chains_interactions[0].keys():
        len_interaction += sum(list_of_chains_interactions[0][chain].values())     
    return len_interaction
        

def clean_information_complexes(df):
    #This function uses the "epitope/paratope_interactions_filtered" and the "epitope/paratope_MWV_dict" columns to create a "epitope/paratope/num_interactions". 
    #This resulting dictionary contains the residue as key (format <aa>_<position_num>_<position_ins>) and the number of times it interacts as value.
    #Note: we should also remove the positions that were only interacting with heteroatoms from the epitope and paratope dictionaries.
    epitope_num_interactions_list = []
    paratope_num_interactions_list = []
    
    epitope_aa_pos_list = []
    paratope_aa_pos_list = []
    
    count_removed_interaction_residues = 0
    
    for index, row in df.iterrows():
        pdb, ag_chain, ab_chain = determine_complex_id(row)
        
        dict_epitope_interactions = row["epitope_interactions_filtered"]
        dict_paratope_interactions = row["paratope_interactions_filtered"]
        dict_epitope_residues = row["epitope_MWV_dict"]
        dict_paratope_residues = row["paratope_MWV_dict"]
        
        new_dict_epitope_num_interactions = {}
        new_dict_paratope_num_interactions = {}
        new_dict_epitope_pos_aa = {}
        new_dict_paratope_pos_aa = {}
        
        if len(ab_chain) == 2: #full length antibody
            #EPITOPE
            #The position_interaction dictionary should be a normal dictionary. The new positon_aminoacid is a nested dictionary of the form {heavy_chain: {}, light_chain: {}. both_chains: {}}
            for i in range(len(ab_chain)):
                dict_interactions_per_chain, dict_pos_aa_per_chain, number_removed_epitope_pos_aa  = make_correct_dictionary(dict_epitope_interactions[ab_chain[i]], dict_epitope_residues[ab_chain[i]])
                new_dict_epitope_pos_aa[ab_chain[i]] = dict_pos_aa_per_chain 
                count_removed_interaction_residues += number_removed_epitope_pos_aa
                
            heavy_dict = dict_epitope_interactions[ab_chain[0]]
            light_dict = dict_epitope_interactions[ab_chain[1]]
            dict_epitope_interactions_num_not_filtered = {k: heavy_dict.get(k, 0) + light_dict.get(k, 0) for k in set(heavy_dict) | set(light_dict)} #sum the number of interactions per position (if epitope residue binds both to heavy and light chain)
            new_dict_epitope_num_interactions, new_dict_epitope_pos_aa_both_chains, number_removed_epitope_pos_aa  = make_correct_dictionary(dict_epitope_interactions_num_not_filtered, dict_epitope_residues[ab_chain])
            new_dict_epitope_pos_aa[ab_chain] = new_dict_epitope_pos_aa_both_chains 
             
            #PARATOPE
            #paratope of full length is in both columns stored as {"heavy_chain": {}, "light_chain": {} }
            for i in range(len(ab_chain)):
                dict_interactions_per_chain, dict_pos_aa_per_chain, number_removed_paratope_pos_aa  = make_correct_dictionary(dict_paratope_interactions[ab_chain[i]], dict_paratope_residues[ab_chain[i]])
                new_dict_paratope_num_interactions[ab_chain[i]] = dict_interactions_per_chain
                new_dict_paratope_pos_aa[ab_chain[i]] = dict_pos_aa_per_chain 
                count_removed_interaction_residues += number_removed_paratope_pos_aa
                
        else: #single domain antibody, stored without nested dictionaries
            new_dict_epitope_num_interactions, new_dict_epitope_pos_aa, number_removed_epitope = make_correct_dictionary(dict_epitope_interactions, dict_epitope_residues)
            new_dict_paratope_num_interactions, new_dict_paratope_pos_aa, number_removed_paratope = make_correct_dictionary(dict_paratope_interactions, dict_paratope_residues)
            count_removed_interaction_residues += (number_removed_epitope + number_removed_paratope)
            
        epitope_num_interactions_list.append(new_dict_epitope_num_interactions)
        paratope_num_interactions_list.append(new_dict_paratope_num_interactions)
        epitope_aa_pos_list.append(new_dict_epitope_pos_aa)
        paratope_aa_pos_list.append(new_dict_paratope_pos_aa)
        
    #Rename old epitope_MWV_dict and paratope_MWV_dict columns; can be removed later after checking!!!! #TO DO
    df.rename(columns={'epitope_MWV_dict':'epitope_MWV_dict_not_filtered'}, inplace=True)
    df.rename(columns={'paratope_MWV_dict':'paratope_MWV_dict_not_filtered'}, inplace=True)
    
    #Add to dataframe
    df['epitope_num_interactions'] = epitope_num_interactions_list #Add the new epitope number interactions dictionary
    df['paratope_num_interactions'] = paratope_num_interactions_list #Add the new paratope number interactions dictionary
    df["epitope_MWV_dict"] = epitope_aa_pos_list
    df["paratope_MWV_dict"] = paratope_aa_pos_list
    
    print(f"Number of removed epitope/paratope residues because only interacting with hetero atom: {count_removed_interaction_residues}")
    
    return df

def make_correct_dictionary(dict_pos_int, dict_pos_aa):
    #This function sets the dictionary of the position and number of interactions in correct format including the corresponding residue which could be found in dict_pos_aa
    #Note: some positions in dict_pos_int do not occur in dict_pos_aa because they were only interacting with hetero atoms. These positions should also be removed from the dict_pos_aa dictionary
    count_not_found_in_previous_epi_para = 0
    new_dict_pos_int = {}
    remove_these_positions = []
    for position in dict_pos_aa.keys():
        try:
            new_key = transform_key_format(position, dict_pos_aa[position])
            new_dict_pos_int[new_key] = dict_pos_int[position]
        except KeyError:
            #If the position can not be found in dict_pos_int, it means that the position was only interacting with an hetero atom. Because these interactions are now removed it cant be found.
            #Remove these positions also from the dict_pos_int
            count_not_found_in_previous_epi_para += 1
            remove_these_positions.append(position)
           
    #Remove the positions after iterating through this dictionary to avoid problems with the size. 
    new_dict_pos_aa = {key: dict_pos_aa[key] for key in dict_pos_aa if key not in remove_these_positions}      
                        
    return new_dict_pos_int, new_dict_pos_aa, count_not_found_in_previous_epi_para

    
def remove_not_found_aa_residues(df, epitope_residues_list, paratope_residues_list):
    #This function removes the residues from the dataframe (removes the rows containing this residue) that are not in the epitope and paratope. 
    #They should be removed as they are determined as interacting but they are hetero atoms.
    
    df = df[df['position_1'].isin(epitope_residues_list)]
    df = df[df['position_2'].isin(paratope_residues_list)]

    return df
    
def store_interactions_residue_per_complex_dict(df_residue_interaction, columns_types_interactions, interaction_types_list):
    #This function stores the interaction of one antigen-antibody complex as one line in a dataframe
    #For all types of bindings that are determined as interactions by "columns_types_interactions". It only counts how often it occurs between two residues. Not how many times it occurs within two residues. 
    #So for example if residues A-B make 3 times a hydrophilic bond (because 3 different atom combinations of these two residues make hydrophilic bonds) it is count as 1 in the summary file. 
    #Note, if these same A-B make also x times another bond this bond is also counted as 1. Therefore one residue pair can make multiple types of interactions 

    dict_interaction_res_occurence_complex = {}
    for interaction_type in columns_types_interactions:
        interaction_seen_between_residues_complex = df_residue_interaction[interaction_type].astype(bool).sum(axis=0)
        dict_interaction_res_occurence_complex[interaction_type] = interaction_seen_between_residues_complex
        
    #Determine epitope, paratope, number interactions, number contacts and add this to the dictionary 
    dict_epitope, dict_paratope, number_interactions, number_contacts = store_epitope_paratope_dict(df_residue_interaction, interaction_types_list)
    dict_interaction_res_occurence_complex["epitope_interactions_filtered"] = [dict_epitope]
    dict_interaction_res_occurence_complex["paratope_interactions_filtered"] = [dict_paratope]
    dict_interaction_res_occurence_complex["number_interactions"] = number_interactions
    dict_interaction_res_occurence_complex["number_contacts"] = number_contacts

    return dict_interaction_res_occurence_complex
    
def store_epitope_paratope_dict(df_residue_interaction, interaction_types_list):
    #This function set up the dataframes and calls the "determine_interaction_dict()" function in order to determine both the epitope and the paratope
    
    #antigen chain always in chain_bgn, antibody chain always in chain_end
    dict_epitope_position, count_number_total_interactions_epi, count_number_total_contacts_epi = determine_interaction_dict(df_residue_interaction, interaction_types_list, "position_1")
    dict_paratope_position, count_number_total_interactions_para, count_number_total_contacts_para = determine_interaction_dict(df_residue_interaction, interaction_types_list, "position_2")
    
    #Sanity check 
    if count_number_total_interactions_epi != count_number_total_interactions_para:
        print("not the same interactions")
    if count_number_total_contacts_epi != count_number_total_contacts_para:
        print("not the same contacts")
        
    return dict_epitope_position, dict_paratope_position, count_number_total_interactions_epi, count_number_total_contacts_epi
    
def determine_interaction_dict(df_residue_interaction, interaction_types_list, column_name):
    #This function determines for every epitope (or paratope) residue how often it is interacting with another residue of the paratope (or epitope). Besides it counts the total amount of interactions within one complex.
    
    interaction_positions_set = set(df_residue_interaction[column_name].tolist())
    dict_interaction_position = {}
    count_number_total_interactions = 0
    count_number_total_contacts = 0
    
    for interaction_pos in interaction_positions_set:
        count_is_interacting = 0
        df_selected_interaction_region = df_residue_interaction.loc[df_residue_interaction[column_name] == interaction_pos]
        
        for index, row in df_selected_interaction_region.iterrows():
            if row[interaction_types_list].sum() >0:
                #If one of the interaction types is seen, count it
                count_is_interacting += 1
                count_number_total_interactions +=1
            else:
                #If no interaction seen, count as contact
                count_number_total_contacts += 1 
        
        #Save with how mamy residues the certain position is interacting. 
        if count_is_interacting > 0:
            dict_interaction_position[interaction_pos] = count_is_interacting
        
    return dict_interaction_position, count_number_total_interactions, count_number_total_contacts
    
    
def transform_key_format(position, amino_acid):
    #make a key in this format: <aa>_<position_num>_<position_ins>
    pos_num, pos_ins = seperate_position_insertion(position)
        
    if pos_ins != " ": #So an insertion does exist 
        new_key = f"{amino_acid}_{pos_num}_{pos_ins}"
    else:
        new_key = f"{amino_acid}_{pos_num}"
    return new_key
    
       
def combine_full_length_output(dict_heavy, dict_light, chain_heavy, chain_light):
    #This function stores the information of the heavy and the light chain seperately and together. 
    dict_combined = {}
    for key in dict_heavy.keys():
        info_dict = {}
        if isinstance(dict_heavy[key], list): #get dictionary out of the list
            dict_heavy[key] = dict_heavy[key][0]
            dict_light[key] = dict_light[key][0]
        info_dict[chain_heavy] = dict_heavy[key]
        info_dict[chain_light] = dict_light[key]
        dict_combined[key] = [info_dict] #Brackets are needed to save it as one entry in the pdb. 
    return dict_combined


In [5]:
#Single domain antibodies
ds_name_nb = "Dataset_nb_filtered.csv"
df_nb = read_dataset(ds_name_nb)

ds_name_arp_nb = "Dataset_nb_filtered_arpeggio.csv"
df_arp_nb = read_dataset(ds_name_arp_nb)

In [6]:
#Full length antibodies
ds_name_fv = "Dataset_fv_filtered.csv"
df_fv = read_dataset(ds_name_fv)

ds_name_arp_fv = "Dataset_fv_filtered_arpeggio.csv"
df_arp_fv = read_dataset(ds_name_arp_fv)

In [7]:
#Single domain
# df_arp_formatted_nb = add_interaction_residues_to_arpeggio_output(df_nb, df_arp_nb)
# df_arp_epi_para_nb = add_residue_interactions_to_arpeggio_output(df_arp_formatted_nb, columns_types_interactions, interaction_types)


In [8]:
# df_arp_epi_para_nb

In [16]:
#Full length antibody
# df_arp_formatted_fv = add_interaction_residues_to_arpeggio_output(df_fv, df_arp_fv)
# df_arp_epi_para_fv = add_residue_interactions_to_arpeggio_output(df_arp_formatted_fv, columns_types_interactions, interaction_types)

Residue 1D code could not be found: 7orb, E, 229
<Residue PO4 het=H_PO4 resseq=229 icode= >
Residue 1D code could not be found: 7orb, L, 234
<Residue TRS het=H_TRS resseq=234 icode= >
Residue 1D code could not be found: 6p67, K, 301
<Residue NAG het=H_NAG resseq=301 icode= >
Residue 1D code could not be found: 3qa3, E, 500
<Residue  CA het=H_ CA resseq=500 icode= >
Residue 1D code could not be found: 6oor, A, 303
<Residue UNL het=H_UNL resseq=303 icode= >
Residue 1D code could not be found: 7tn0, S, 602
<Residue EDO het=H_EDO resseq=602 icode= >
Residue 1D code could not be found: 7tn0, M, 224
<Residue EDO het=H_EDO resseq=224 icode= >
Residue 1D code could not be found: 7tn0, S, 605
<Residue NAG het=H_NAG resseq=605 icode= >
Residue 1D code could not be found: 7dc8, B, 216
<Residue ATP het=H_ATP resseq=216 icode= >
Residue 1D code could not be found: 7bej, E, 803
<Residue FMT het=H_FMT resseq=803 icode= >
Residue 1D code could not be found: 4qhu, A, 231
<Residue GOL het=H_GOL resseq=2

Residue 1D code could not be found: 7kd6, O, 134
<Residue SO4 het=H_SO4 resseq=134 icode= >
Residue 1D code could not be found: 2xqb, H, 232
<Residue SO4 het=H_SO4 resseq=232 icode= >
Residue 1D code could not be found: 4ydl, A, 504
<Residue NAG het=H_NAG resseq=504 icode= >
Residue 1D code could not be found: 4k9e, C, 601
<Residue NAG het=H_NAG resseq=601 icode= >
Residue 1D code could not be found: 4zso, F, 303
<Residue FUC het=H_FUC resseq=303 icode= >
Residue 1D code could not be found: 3ld8, A, 337
<Residue GOL het=H_GOL resseq=337 icode= >
Residue 1D code could not be found: 3hi6, A, 1
<Residue  MN het=H_ MN resseq=1 icode= >
Residue 1D code could not be found: 4j4p, B, 601
<Residue NAG het=H_NAG resseq=601 icode= >
Residue 1D code could not be found: 7coe, H, 225
<Residue EDO het=H_EDO resseq=225 icode= >
Residue 1D code could not be found: 7qu2, C, 302
<Residue GOL het=H_GOL resseq=302 icode= >
Residue 1D code could not be found: 3v6o, B, 1002
<Residue EDO het=H_EDO resseq=1002

In [17]:
df_arp_epi_para_fv

,pdb,antigen_chain,antibody_chain,epitope_MWV_dict,epitope_num_interactions,paratope_MWV_dict,paratope_num_interactions,number_interactions,number_contacts,clash,...,hbond,weak_hbond,xbond,ionic,metal_complex,aromatic,hydrophobic,carbonyl,polar,weak_polar
0,7mdj,C,AB,"{'A': {'48': 'V', '52': 'R', '62': 'Y', '45': ...","{'V_48': 1, 'R_52': 3, 'Y_62': 4, 'Y_45': 1, '...","{'A': {'110': 'D', '109': 'G', '108': 'R', '62...","{'A': {'D_110': 1, 'G_109': 2, 'R_108': 1, 'Y_...","{'A': 14, 'B': 14}","{'A': 7, 'B': 6}","{'A': 0, 'B': 0}",...,"{'A': 5, 'B': 4}","{'A': 3, 'B': 4}","{'A': 0, 'B': 0}","{'A': 2, 'B': 1}","{'A': 0, 'B': 0}","{'A': 2, 'B': 0}","{'A': 6, 'B': 5}","{'A': 0, 'B': 2}","{'A': 5, 'B': 5}","{'A': 6, 'B': 6}"
1,7sem,F,BC,"{'B': {'282': 'P', '275': 'I', '314': 'A', '28...","{'P_282': 1, 'I_275': 1, 'A_314': 1, 'T_281': ...","{'B': {'110': 'T', '59': 'S', '57': 'S', '62':...","{'B': {'T_110': 1, 'S_59': 1, 'S_57': 1, 'S_62...","{'B': 23, 'C': 6}","{'B': 10, 'C': 1}","{'B': 0, 'C': 0}",...,"{'B': 4, 'C': 2}","{'B': 7, 'C': 2}","{'B': 0, 'C': 0}","{'B': 0, 'C': 0}","{'B': 0, 'C': 0}","{'B': 0, 'C': 0}","{'B': 7, 'C': 1}","{'B': 1, 'C': 0}","{'B': 9, 'C': 4}","{'B': 8, 'C': 1}"
2,7jmo,A,HL,"{'H': {'489': 'Y', '457': 'R', '473': 'Y', '48...","{'Y_489': 2, 'R_457': 1, 'Y_473': 1, 'F_486': ...","{'H': {'110': 'R', '109': 'E', '108': 'L', '59...","{'H': {'R_110': 2, 'E_109': 3, 'L_108': 3, 'G_...","{'H': 31, 'L': 8}","{'H': 17, 'L': 3}","{'H': 0, 'L': 0}",...,"{'H': 9, 'L': 3}","{'H': 12, 'L': 0}","{'H': 0, 'L': 0}","{'H': 1, 'L': 0}","{'H': 0, 'L': 0}","{'H': 0, 'L': 1}","{'H': 13, 'L': 2}","{'H': 0, 'L': 0}","{'H': 13, 'L': 4}","{'H': 10, 'L': 0}"
3,7orb,X,EF,"{'E': {'448': 'N', '444': 'K', '447': 'G', '49...","{'N_448': 2, 'K_444': 4, 'G_447': 1, 'L_492': ...","{'E': {'110': 'V', '111B': 'A', '112A': 'Y', '...","{'E': {'V_110': 1, 'A_111_B': 3, 'Y_112_A': 1,...","{'E': 16, 'F': 11}","{'E': 6, 'F': 2}","{'E': 0, 'F': 0}",...,"{'E': 4, 'F': 4}","{'E': 5, 'F': 2}","{'E': 0, 'F': 0}","{'E': 0, 'F': 0}","{'E': 0, 'F': 0}","{'E': 0, 'F': 1}","{'E': 5, 'F': 5}","{'E': 1, 'F': 0}","{'E': 7, 'F': 5}","{'E': 6, 'F': 4}"
4,7orb,R,HL,"{'H': {'456': 'F', '478': 'T', '489': 'Y', '47...","{'F_456': 1, 'T_478': 2, 'Y_489': 2, 'Y_473': ...","{'H': {'62': 'S', '57': 'V', '107': 'P', '113'...","{'H': {'S_62': 1, 'V_57': 1, 'P_107': 1, 'D_11...","{'H': 14, 'L': 3}","{'H': 7, 'L': 0}","{'H': 0, 'L': 0}",...,"{'H': 5, 'L': 0}","{'H': 3, 'L': 0}","{'H': 0, 'L': 0}","{'H': 0, 'L': 0}","{'H': 0, 'L': 0}","{'H': 1, 'L': 1}","{'H': 6, 'L': 2}","{'H': 1, 'L': 0}","{'H': 4, 'L': 1}","{'H': 2, 'L': 0}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,4lst,G,HL,"{'H': {'282': 'K', '457': 'D', '365': 'S', '28...","{'K_282': 1, 'D_457': 1, 'S_365': 1, 'A_281': ...","{'H': {'65': 'V', '63': 'G', '112': 'N', '111'...","{'H': {'V_65': 1, 'G_63': 1, 'N_112': 1, 'D_11...","{'H': 28, 'L': 11}","{'H': 9, 'L': 2}","{'H': 0, 'L': 0}",...,"{'H': 11, 'L': 4}","{'H': 5, 'L': 0}","{'H': 0, 'L': 0}","{'H': 3, 'L': 1}","{'H': 0, 'L': 0}","{'H': 0, 'L': 0}","{'H': 7, 'L': 4}","{'H': 0, 'L': 0}","{'H': 12, 'L': 5}","{'H': 8, 'L': 2}"
886,4lss,G,HL,"{'H': {'282': 'K', '457': 'D', '281': 'A', '45...","{'K_282': 1, 'D_457': 3, 'A_281': 4, 'G_458': ...","{'H': {'70': 'P', '63': 'G', '112': 'N', '111'...","{'H': {'P_70': 1, 'G_63': 1, 'N_112': 1, 'D_11...","{'H': 34, 'L': 5}","{'H': 7, 'L': 1}","{'H': 0, 'L': 0}",...,"{'H': 12, 'L': 2}","{'H': 6, 'L': 0}","{'H': 0, 'L': 0}","{'H': 4, 'L': 0}","{'H': 0, 'L': 0}","{'H': 0, 'L': 0}","{'H': 10, 'L': 2}","{'H': 1, 'L': 0}","{'H': 13, 'L': 2}","{'H': 6, 'L': 1}"
887,2yss,C,BA,"{'B': {'73': 'R', '77': 'N', '102': 'G', '101'...","{'R_73': 1, 'N_77': 1, 'G_102': 2, 'D_101': 5,...","{'B': {'59': 'S', '108': 'D', '57': 'S', '107'...","{'B': {'S_59': 2, 'D_108': 1, 'S_57': 1, 'W_10...","{'B': 25, 'A': 14}","{'B': 8, 'A': 3}","{'B': 0, 'A': 0}",...,"{'B': 8, 'A': 6}","{'B': 9, 'A': 2}","{'B': 0, '

In [18]:
##Save
# df_arp_epi_para_nb.to_csv("Dataset_nb_arpeggio_interactions.csv", index=False)
# df_arp_epi_para_fv.to_csv("Dataset_fv_arpeggio_interactions.csv", index = False)